In [ ]:
import generator
import time

import numpy as np

from pysat.formula import CNF 
from pysat.solvers import Solver


def read_sat(sat_path):
    with open(sat_path) as f:
        sat_lines = f.readlines()
        header = sat_lines[0]
        header_info = header.replace("\n", "").split(" ")
        num_vars = int(header_info[-2])
        num_clauses = int(header_info[-1])

        sat = [[int(x) for x in line.replace(' 0\n', '').split(' ')]
               for line in sat_lines[1:]]

        return sat, num_vars, num_clauses


def sat_transform(sat):
    new_sat = [set([abs(x) for x in clause]) for clause in sat]
    return new_sat


def if_dependent(x, y):
    set_x = set(abs(i) for i in x)
    set_y = set(abs(i) for i in y)
    if set_x.intersection(set_y):
        return True
    else:
        return False


def dependency_analysis(sat, num_vars, num_clauses):
    dependency = [[] for i in range(num_clauses)]

    start_time = time.time()
    inersection_graph = np.zeros([num_clauses, num_vars])
    for idx, clause in enumerate(sat):
        for literal in clause:
            var_idx = abs(literal) - 1
            if literal > 0:
                inersection_graph[idx, var_idx] = 1
            else:
                inersection_graph[idx, var_idx] = -1
    print(f"inersection_graph build time: {time.time() - start_time:.4f}")

    for i in range(num_clauses):
        for j in range(i+1, num_clauses):
            pair_dependency = np.multiply(
                inersection_graph[i], inersection_graph[j])
            if np.any(pair_dependency):
                dependency[i].append(j)
                dependency[j].append(i)

    return dependency


def lll_test(sat, n, k, log=False): # n is the varible number in sat
    num_clauses = len(sat)
    # initial the alpahs for each clause (event)
    event_prob = [2 ** (-len(clause)) for clause in sat]
    # print(f'initial alphas: {alphas}')
    # dependency analysis
    start_time = time.time()
    a = [[] for i in range(n + 1)] # j in a[i] means varible x_i is in j-th clauses
    dependency = [[] for i in range(num_clauses)]
    
    for i in range(num_clauses):
        clause_len = len(sat[i])
        for j in range(clause_len):
            a[abs(sat[i][j])].append(i)
            
    for i in range(1, n + 1):
        len_i = len(a[i])
        if len_i <= 1: continue
        for j in range(len_i):
            for l in range(j + 1, len_i):
                dependency[a[i][j]].append(a[i][l])
                dependency[a[i][l]].append(a[i][j])
                
    for i in range(num_clauses):
        dependency[i] = np.unique(dependency[i])
    """ 
    sat = sat_transform(sat)
    dependency = [[] for i in range(num_clauses)]
    for i in range(num_clauses):
        for j in range(i+1, num_clauses):
            if sat[i].intersection(sat[j]):
                dependency[i].append(j)
                dependency[j].append(i)
    """
    # print(f"dependency analysis time: {time.time() - start_time:.4f}")

    # propagation
    iterations = 0
    alphas = [x for x in event_prob]
    for i in range(k):
        if log:
            print(alphas)
        iterations += 1
        if max(alphas) > 1:
            break
        
        update_alphas = [event_prob[i] / np.prod([1 - alphas[j] for j in dependency[i]]) for i in range(num_clauses)]
        alphas = update_alphas

    return iterations, dependency


In [ ]:
from pysat.formula import CNF
from pysat.solvers import Solver

n = 10000 # variable number
sat_instance = generator.lll_sat_generator(n, 5)

#sat_instance = [[1, -2, 3], [1, 2, -3], [1, 2, 3]]
# sat_instance = [[1, -2, 3]]


# sat_path = './industrial_formulas/cmu-bmc-longmult15.processed.cnf'
# sat_instance, num_vars, num_clauses = read_sat(sat_path)


print(f'the size of SAT: {len(sat_instance)}')

print('----------')

iterations, dependency = lll_test(sat_instance, n, 100, log=False)
print(f'lll_test iterations: {iterations}')

print('----------')

cnf = CNF(from_clauses=sat_instance)
start_time = time.time()
with Solver(bootstrap_with=cnf) as solver:
    print(f'The SAT instance can be solved: {solver.solve()}')
    # print(solver.get_model())
print(f"solver time: {time.time() - start_time:.4f}")

In [ ]:
import os

formulas_path = './formulas/'

for path in os.listdir(formulas_path):
    sat_path = formulas_path + path
    sat_instance, num_vars, num_clauses = read_sat(sat_path)
    iterations, dependency = lll_test(sat_instance, 100, 20)
    if iterations > 2:
        print('<-------->')
        print(path)
        print(f'lll_test iterations: {iterations}')
    

In [ ]:
unsolve_path = './unsolve_formulas/'

unsolve_instances = []


for path in os.listdir(formulas_path):
    sat_path = formulas_path + path
    sat_instance, num_vars, num_clauses = read_sat(sat_path)
    unsolve_instances.append(sat_instance)


plot_data = []

num_all = len(unsolve_instances)
for i in range(1, 100):
    num_lll = 0
    part_instances = [instance[:i] for instance in unsolve_instances]
    for instance in part_instances:
        iterations, dependency = lll_test(instance, 100, 20)
        if iterations == 20:
            num_lll += 1

    print(f'sat_size: {i}, number of all: {num_all}, number of lll: {num_lll}, percent: {num_lll/num_all}')
    plot_data.append((i/100, num_lll/num_all))
    

In [ ]:
instances_sizes = []

for path in os.listdir(formulas_path):
    sat_path = formulas_path + path
    sat_instance, num_vars, num_clauses = read_sat(sat_path)
    unsolve_instances.append(sat_instance)
    instances_sizes.append(num_clauses)

print(len(instances_sizes))
max_size = max(instances_sizes)
print(max_size)

instance_distribution = [[], []]
for i in range(1, max_size+1):
    instance_distribution[0].append(i/100)
    instance_distribution[1].append(1 - len([x for x in instances_sizes if x < i])/len(instances_sizes))

print(instance_distribution)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axisartist.axislines import AxesZero



fig = plt.figure()

ax = fig.add_subplot(axes_class=AxesZero)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1.1)

for direction in ["xzero", "yzero"]:
    # adds arrows at the ends of each axis
    ax.axis[direction].set_axisline_style("-|>")

    # adds X and Y-axis from the origin
    ax.axis[direction].set_visible(True)

for direction in ["left", "right", "bottom", "top"]:
    # hides borders
    ax.axis[direction].set_visible(False)

ax.plot(*zip(*plot_data), color='blue')

# ax.plot(instance_distribution[0], instance_distribution[1], color='red')

ax.set_xlabel("clause density")
ax.set_ylabel("satisfiable rate")

fig.show()
fig.savefig('lll-pt.png')

In [ ]:
import matplotlib.pyplot as plt

lll_data = [x for x in plot_data]

lll_data.append((14, 0))

fig = plt.figure()

ax = fig.add_subplot(axes_class=AxesZero)

ax.set_xlim(0, 16)
ax.set_ylim(0, 1.1)

for direction in ["xzero", "yzero"]:
    # adds arrows at the ends of each axis
    ax.axis[direction].set_axisline_style("-|>")

    # adds X and Y-axis from the origin
    ax.axis[direction].set_visible(True)

for direction in ["left", "right", "bottom", "top"]:
    # hides borders
    ax.axis[direction].set_visible(False)


ax.plot(*zip(*lll_data), color='blue')

ax.plot(instance_distribution[0], instance_distribution[1], color='red')

ax.set_xlabel("clause density")
ax.set_ylabel("satisfiable rate")

fig.show()
fig.savefig('lll-satisfiable-pt.png')